### Entegra Report Analysis

In [1755]:
# pip install geopy

In [1756]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

##### Functions

In [1757]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [1758]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [1759]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [1760]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [1761]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [1762]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [1763]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [1764]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [1765]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [1766]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

#### Report

In [1767]:
input_path = r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Sales"
output_path = r"C:\Users\user\Downloads"

data_file = r"Entegra_Sales_List (01.01.2024-11.07.2024)"

In [1768]:
# df_entegra = pd.read_excel(fr"{input_path}\{data_file}.xlsx")

# df_entegra.to_csv(fr"{output_path}\{data_file}.csv", encoding='utf8', index=False)

In [1769]:
df_entegra = pd.read_csv(fr"{input_path}\{data_file}.csv", low_memory=False)
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productCode,payment_type,total_product,PazaryerindenGelenOdemeTutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,FaturaTipiAdi
0,178856,2280224850,9352213652,NaN,2024-07-11 16:21:44,2024-07-11 16:23:02,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
1,178857,2280223780,9352211863,NaN,2024-07-11 16:20:54,2024-07-11 16:23:17,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
2,178858,2280223782,9352210132,NaN,2024-07-11 16:20:05,2024-07-11 16:23:19,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
3,178859,2280222421,9352209819,NaN,2024-07-11 16:19:57,2024-07-11 16:23:22,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,2,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
4,178860,2280220893,9352209340,NaN,2024-07-11 16:19:42,2024-07-11 16:23:24,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,83601,207423367318-1,207423367318,NaN,2024-01-01 00:18:00,2024-01-01 00:24:34,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz
95055,83598,1935764998,2424519644,NaN,2024-01-01 00:14:43,2024-01-01 00:18:56,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,3,Ertesi Gün Kargoda,2024-01-02 06:41:47,NaN,0,Seciniz
95056,83596,1935755636,8061482665,NaN,2024-01-01 00:02:03,2024-01-01 00:04:53,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,2024-01-02 08:31:11,NaN,0,Seciniz
95057,83597,201757367316-1,201757367316,NaN,2024-01-01 00:02:00,2024-01-01 00:07:04,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz


In [1770]:
df_entegra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95059 entries, 0 to 95058
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            95059 non-null  int64  
 1   order_number                  95059 non-null  object 
 2   platform_reference_no         95059 non-null  object 
 3   invoice_number                0 non-null      float64
 4   datetime                      95059 non-null  object 
 5   date_add                      95059 non-null  object 
 6   entegration                   95059 non-null  object 
 7   supplier                      95059 non-null  object 
 8   status                        95059 non-null  int64  
 9   status_name                   95059 non-null  object 
 10  company                       95058 non-null  object 
 11  cargo_code                    92937 non-null  object 
 12  invoice_country               2849 non-null   object 
 13  i

In [1771]:
df_entegra.columns = [x.lower() for x in df_entegra.columns]
df_entegra.columns = [x.strip() for x in df_entegra.columns]
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productcode,payment_type,total_product,pazaryerindengelenodemetutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,faturatipiadi
0,178856,2280224850,9352213652,NaN,2024-07-11 16:21:44,2024-07-11 16:23:02,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
1,178857,2280223780,9352211863,NaN,2024-07-11 16:20:54,2024-07-11 16:23:17,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
2,178858,2280223782,9352210132,NaN,2024-07-11 16:20:05,2024-07-11 16:23:19,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
3,178859,2280222421,9352209819,NaN,2024-07-11 16:19:57,2024-07-11 16:23:22,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,2,Ertesi Gün Kargoda,NaN,NaN,0,Seciniz
4,178860,2280220893,9352209340,NaN,2024-07-11 16:19:42,2024-07-11 16:23:24,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,NaN,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95054,83601,207423367318-1,207423367318,NaN,2024-01-01 00:18:00,2024-01-01 00:24:34,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz
95055,83598,1935764998,2424519644,NaN,2024-01-01 00:14:43,2024-01-01 00:18:56,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,3,Ertesi Gün Kargoda,2024-01-02 06:41:47,NaN,0,Seciniz
95056,83596,1935755636,8061482665,NaN,2024-01-01 00:02:03,2024-01-01 00:04:53,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,2024-01-02 08:31:11,NaN,0,Seciniz
95057,83597,201757367316-1,201757367316,NaN,2024-01-01 00:02:00,2024-01-01 00:07:04,N11,n11,1,Yeni Siparis,...,160620230178,N11,1,0,1,NaN,2024-01-02 10:46:02,NaN,0,Seciniz


In [1772]:
df_entegra.columns

Index(['id', 'order_number', 'platform_reference_no', 'invoice_number',
       'datetime', 'date_add', 'entegration', 'supplier', 'status',
       'status_name', 'company', 'cargo_code', 'invoice_country',
       'invoice_city', 'tax_office', 'tax_number', 'tc_id', 'total', 'tax',
       'grand_total', 'sync', 'erp_order_number', 'erp_message',
       'store_order_status', 'store_order_status_name', 'order_process',
       'note', 'supplier1', 'supplier_id', 'cargo_sync', 'cargo_company',
       'cargo_code2', 'cargo_follow_url', 'cargo_finally_statu',
       'cargo_error_message', 'einvoice_error_message', 'invoice_print_name',
       'ship_print_name', 'sevk_tarihi', 'ship_country', 'customer_code',
       'cargo_fee_type_name', 'cargo_packet_type', 'cargo_packet_quantity',
       'grand_total_desi', 'model', 'product_name', 'store_product_name',
       'invoice_name', 'pov_productcode', 'payment_type', 'total_product',
       'pazaryerindengelenodemetutar', 'total_product_quantity',

In [1773]:
df_entegra_select_var = df_entegra[["model","product_name","datetime","total_product_quantity","entegration",\
    "invoice_city","invoice_country","total","tax","grand_total","cargo_company","status_name"]]
df_entegra_select_var

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,NaN,383.33,76.67,460.0,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,NaN,154.17,30.83,185.0,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,NaN,298.33,59.67,358.0,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,NaN,300.00,60.00,360.0,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,NaN,383.33,76.67,460.0,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
95054,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,NaN,153.64,15.36,169.0,mng,Yeni Siparis
95055,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,NaN,318.18,31.82,350.0,trendyolekspress,Onaylandı
95056,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,NaN,312.50,62.50,375.0,trendyolekspress,Onaylandı
95057,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,NaN,153.64,15.36,169.0,mng,Yeni Siparis


In [1774]:
df_entegra_select_var["invoice_country"].unique()

array([nan, 'Suudi Arabistan', 'Azerbaycan', 'Birleşik Arap Emirlikleri',
       'Türkiye', 'Bahreyn'], dtype=object)

In [1776]:
df_entegra_select_var.loc[:,"invoice_country"] = df_entegra_select_var.loc[:,"invoice_country"].fillna("Türkiye")
df_entegra_select_var

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
95054,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,mng,Yeni Siparis
95055,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,trendyolekspress,Onaylandı
95056,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,Türkiye,312.50,62.50,375.0,trendyolekspress,Onaylandı
95057,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,mng,Yeni Siparis


In [1778]:
df_entegra_select_var[df_entegra_select_var["grand_total"] == df_entegra_select_var["grand_total"].max()]

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
21396,004375,AWOX Vıte Pe330 Kıyma Makinesi,2024-06-03 14:38:06,40,Woocommerce,Bilecik,Türkiye,100000.0,20000.0,120000.0,NaN,Yeni Siparis


In [1780]:
df_entegra_select_var[df_entegra_select_var.duplicated(keep=False)].head(5)

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
4005,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,hepsijet,Onaylandı
4006,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,hepsijet,Onaylandı
4007,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-03 23:01:51,3,HepsiBurada,İstanbul,Türkiye,316.6583,63.3317,379.99,hepsijet,Onaylandı
4048,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-03 21:25:21,1,HepsiBurada,Mardin,Türkiye,383.3250,76.6650,459.99,hepsijet,Onaylandı
4049,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-03 21:25:21,1,HepsiBurada,Mardin,Türkiye,383.3250,76.6650,459.99,eptt,Onaylandı


In [1781]:
df_entegra_select_var[df_entegra_select_var.duplicated(keep=False)].to_excel(fr"{output_path}\Entegra Aynı Sipariş Tekrarlama.xlsx", index=False)

In [1782]:
# pd.to_datetime(df_entegra_select_var['datetime'], format='%Y-%m-%d %H:%M:%S')

In [1783]:
df_entegra_select_var = df_entegra_select_var.drop_duplicates()
df_entegra_select_var = df_entegra_select_var.reset_index(drop=True)
df_entegra_select_var

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,mng,Yeni Siparis
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,trendyolekspress,Onaylandı
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,trendyol,Aydın,Türkiye,312.50,62.50,375.0,trendyolekspress,Onaylandı
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,mng,Yeni Siparis


In [1784]:
df_entegra_select_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94834 entries, 0 to 94833
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   model                   94833 non-null  object 
 1   product_name            94833 non-null  object 
 2   datetime                94834 non-null  object 
 3   total_product_quantity  94834 non-null  int64  
 4   entegration             94834 non-null  object 
 5   invoice_city            94833 non-null  object 
 6   invoice_country         94834 non-null  object 
 7   total                   94834 non-null  float64
 8   tax                     94834 non-null  float64
 9   grand_total             94834 non-null  float64
 10  cargo_company           94834 non-null  object 
 11  status_name             94834 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 8.7+ MB


In [1785]:
df_entegra_select_var.loc[:,'datetime'] = pd.to_datetime(df_entegra_select_var.loc[:,'datetime'], format='%Y-%m-%d %H:%M:%S')
df_entegra_select_var.head(5)

,model,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı


In [1786]:
pd.to_datetime(df_entegra_select_var["datetime"]).dt.date.head(5)

0    2024-07-11
1    2024-07-11
2    2024-07-11
3    2024-07-11
4    2024-07-11
Name: datetime, dtype: object

In [1787]:
pd.to_datetime(df_entegra_select_var["datetime"]).dt.time.head(5)

0    16:21:44
1    16:20:54
2    16:20:05
3    16:19:57
4    16:19:42
Name: datetime, dtype: object

In [1788]:
df_entegra_select_var.insert(3,"date",pd.to_datetime(df_entegra_select_var["datetime"]).dt.date)
df_entegra_select_var.insert(4,"time",pd.to_datetime(df_entegra_select_var["datetime"]).dt.time)
df_entegra_select_var.head(5)

,model,product_name,datetime,date,time,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,trendyol,Yalova,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,trendyol,Bolu,Türkiye,154.17,30.83,185.0,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,trendyol,Ağrı,Türkiye,298.33,59.67,358.0,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,trendyol,İstanbul,Türkiye,300.00,60.00,360.0,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,trendyol,Samsun,Türkiye,383.33,76.67,460.0,trendyolekspress,Onaylandı


In [1790]:
object_columns = df_entegra_select_var.select_dtypes(include='object')
object_columns

,model,product_name,datetime,date,time,entegration,invoice_city,invoice_country,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,trendyol,Yalova,Türkiye,trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,trendyol,Bolu,Türkiye,trendyolekspress,Onaylandı
2,003954,AWOX Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,trendyol,Ağrı,Türkiye,trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,trendyol,İstanbul,Türkiye,trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,trendyol,Samsun,Türkiye,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,N11,Kütahya,Türkiye,mng,Yeni Siparis
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,trendyol,Adıyaman,Türkiye,trendyolekspress,Onaylandı
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,trendyol,Aydın,Türkiye,trendyolekspress,Onaylandı
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,N11,Kocaeli,Türkiye,mng,Yeni Siparis


In [1791]:
object_columns.columns

Index(['model', 'product_name', 'datetime', 'date', 'time', 'entegration',
       'invoice_city', 'invoice_country', 'cargo_company', 'status_name'],
      dtype='object')

In [1792]:
for column in object_columns.columns:
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: convert_one_character_letter(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: whitespace_del(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: title_func(x))
    
df_entegra_select_var.head(10)

,model,product_name,datetime,date,time,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.00,Trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.00,Trendyolekspress,Onaylandı
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.00,Trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.00,Trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.00,Trendyolekspress,Onaylandı
5,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11 16:18:35,2024-07-11,16:18:35,1,Trendyol,Ankara,Türkiye,366.67,73.33,440.00,Trendyolekspress,Onaylandı
6,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:17:52,2024-07-11,16:17:52,1,Trendyol,Eskişehir,Türkiye,154.17,30.83,185.00,Trendyolekspress,Onaylandı
7,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:13:44,2024-07-11,16:13:44,1,Trendyol,Buraidah,Suudi Arabistan,460.00,0.00,460.00,Trendyolekspress,Onaylandı
8,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe,2024-07-11 16:11:43,2024-07-11,16:11:43,1,Trendyol,Sivas,Türkiye,20.83,4.17,24.99,Mng,Onaylandı
9,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:09:44,2024-07-11,16:09:44,1,Trendyol,İstanbul,Türkiye,383.33,76.67,460.00,Trendyolekspress,Onaylandı


In [1796]:
df_entegra_select_product = df_entegra_select_var[["model","product_name"]]
df_entegra_select_product

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal..."
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal..."


In [1797]:
df_entegra_select_product["model"].nunique()

1045

In [1798]:
df_entegra_select_product["product_name"].nunique()

1084

In [1799]:
df_entegra_select_product[df_entegra_select_product.duplicated()]

,model,product_name
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
6,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
7,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
9,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal..."
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal..."


In [1800]:
df_entegra_select_product = df_entegra_select_product.drop_duplicates()
df_entegra_select_product

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
5,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
8,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
94292,002374,Da69 Ahşap Resim Şovalesi 37 Cm
94584,002421,"Çg65 Ahşap Çocuk Makyaj Masası, Pembe Makyaj M..."
94596,002061,Müller Di̇zel Enjektör Temizleyi̇ci̇ (Üstün Al...
94612,002792,21512-21010 Karter Tapasi Kisa Universal


In [1801]:
# df_entegra_select_product.to_excel(fr"{output_path}\Entegra Model Product Name.xlsx", index=False)  # Adoba..Data Folder

##### Geo Location

In [1802]:
def reverse_geocoding(lat, lon):
    '''This function is used for giving city or country name from longitude latitude value \n
    reverse_geocoding(lat, lon): lat is geographical lattitude and lon is geographical longitude \n
    ex. \n
    df['address'] = np.vectorize(reverse_geocoding)(df['Latitude'], df['Longitude'])'''
    try:
        location = geolocator.reverse(Point(lat, lon))
        return location.raw['display_name']
    except:
        return None

In [1803]:
def findGeocode(city):
    '''This function is used for giving latitude and longitude values from city or country name \n
    findGeocode(city): city is a city or country \n
    ex. \n
    loc = findGeocode(i) \n 
    lat = loc.latitude \n
    lon = loc.longitude''' 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    try: 
          
        # Specify the user_agent as your 
        # app name it should not be none 
        geolocator = Nominatim(user_agent="tesst") 
          
        return geolocator.geocode(city) 
      
    except GeocoderTimedOut: 
          
        return findGeocode(city)     

In [1804]:
df_entegra_select_var["invoice_city"].nunique()

237

In [1805]:
# df_entegra_select_var["invoice_city"].unique()

In [1806]:
df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"]

,model,product_name,datetime,date,time,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
20994,Nan,Nan,2024-06-03 22:00:13,2024-06-03,22:00:13,0,Woocommerce,Nan,Türkiye,0.0,0.0,0.0,Diger,Yeni Siparis


In [1807]:
nan_index = df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"].index[0]
nan_index

20994

In [1808]:
# df_nan_var = df_entegra_select_var[df_entegra_select_var["invoice_city"] == "Nan"]
# df_nan_var

In [1809]:
# df_nan_var.loc[nan_index,"invoice_city"] = "İstanbul"

In [1810]:
df_entegra_city_select = df_entegra_select_var[["invoice_city", "invoice_country"]]
df_entegra_city_select = df_entegra_city_select.drop_duplicates(subset="invoice_city")
df_entegra_city_select = df_entegra_city_select.reset_index(drop=True)
df_entegra_city_select.head(5)

,invoice_city,invoice_country
0,Yalova,Türkiye
1,Bolu,Türkiye
2,Ağrı,Türkiye
3,İstanbul,Türkiye
4,Samsun,Türkiye


In [1811]:
# geolocator = Nominatim(user_agent="test")
# 
# # each value from city column 
# # will be fetched and sent to 
# # function find_geocode  
# city = [] 
# latitude = [] 
# longitude = []
# 
# for i in (df_entegra_city_select["invoice_city"]): 
#       
#     if findGeocode(i) != None: 
#            
#         loc = findGeocode(i) 
#           
#         # coordinates returned from  
#         # function is stored into 
#         # two separate list 
#         city.append(i)
#         latitude.append(loc.latitude) 
#         longitude.append(loc.longitude) 
#        
#     # if coordinate for a city not 
#     # found, insert "NaN" indicating  
#     # missing value  
#     else:
#         city.append(np.nan) 
#         latitude.append(np.nan) 
#         longitude.append(np.nan)

In [1812]:
# df_entegra_city_select["latitude"] = latitude
# df_entegra_city_select["longitude"] = longitude 
# df_entegra_city_select

In [1813]:
# df_entegra_city_select.to_excel(fr"{output_path}\Entegra City Location.xlsx", index=False)  # Adoba..Data Folder

##### Entegra Select And Geo Location

In [1814]:
df_city_location = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Geo Location\Entegra City Location.xlsx")
df_city_location.drop("id", axis=1, inplace=True)
df_city_location

,city,country,invoice_city,latitude,longitude
0,Adana,Turkey,Adana,37.143800,35.498409
1,Adiyaman,Turkey,Adıyaman,37.763750,38.276348
2,Adiyaman,Turkey,Adiyaman,37.763750,38.276348
3,Afyonkarahisar,Turkey,Afyonkarahisar,38.685273,30.642741
4,Afyonkarahisar,Turkey,Afyon,38.685273,30.642741
...,...,...,...,...,...
262,Yanbu,Saudi Arabia,Yanbu,24.252616,38.253662
263,Zulfi,Saudi Arabia,Zulfi,26.312990,44.834462
264,Qairawan,Tunisia,Qairawan,35.673549,10.098636
265,Al Khodh,Oman,Al Khodh,23.565997,58.118169


In [1815]:
# df_city_location[df_city_location["city"].duplicated(keep=False)]

In [1816]:
df_city_location["city"].nunique()

195

In [1817]:
df_entegra_select_var

,model,product_name,datetime,date,time,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Trendyolekspress,Onaylandı
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Trendyolekspress,Onaylandı
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Trendyolekspress,Onaylandı
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Trendyolekspress,Onaylandı
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Mng,Yeni Siparis
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Trendyolekspress,Onaylandı
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Trendyolekspress,Onaylandı
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Mng,Yeni Siparis


In [1818]:
df_entegra_city_location = pd.merge(df_entegra_select_var, df_city_location, how="left", on="invoice_city")
df_entegra_city_location = df_entegra_city_location.drop_duplicates().reset_index(drop=True)
df_entegra_city_location

,model,product_name,datetime,date,time,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Yalova,Turkey,40.592898,29.077612
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Trendyolekspress,Onaylandı,Bolu,Turkey,40.621210,31.646026
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Trendyolekspress,Onaylandı,Agri,Turkey,39.777918,43.168437
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Mng,Yeni Siparis,Kütahya,Turkey,39.252251,29.493773
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Trendyolekspress,Onaylandı,Adiyaman,Turkey,37.763750,38.276348
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Trendyolekspress,Onaylandı,Aydın,Turkey,37.837948,27.845691
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Mng,Yeni Siparis,Kocaeli,Turkey,40.821654,29.950718


In [1819]:
df_entegra_select = df_entegra_city_location.drop(["invoice_city","invoice_country"], axis=1)
df_entegra_select

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Yalova,Turkey,40.592898,29.077612
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,154.17,30.83,185.0,Trendyolekspress,Onaylandı,Bolu,Turkey,40.621210,31.646026
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,298.33,59.67,358.0,Trendyolekspress,Onaylandı,Agri,Turkey,39.777918,43.168437
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,300.00,60.00,360.0,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94829,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kütahya,Turkey,39.252251,29.493773
94830,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,3,Trendyol,318.18,31.82,350.0,Trendyolekspress,Onaylandı,Adiyaman,Turkey,37.763750,38.276348
94831,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,1,Trendyol,312.50,62.50,375.0,Trendyolekspress,Onaylandı,Aydın,Turkey,37.837948,27.845691
94832,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kocaeli,Turkey,40.821654,29.950718


In [1820]:
df_entegra_select["latitude"] = df_entegra_select["latitude"].apply(lambda x: round(x, 6))
df_entegra_select["longitude"] = df_entegra_select["longitude"].apply(lambda x: round(x, 6))
df_entegra_select.head(5)

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Yalova,Turkey,40.592898,29.077612
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,154.17,30.83,185.0,Trendyolekspress,Onaylandı,Bolu,Turkey,40.621210,31.646026
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,298.33,59.67,358.0,Trendyolekspress,Onaylandı,Agri,Turkey,39.777918,43.168437
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,300.00,60.00,360.0,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060


In [1821]:
print(df_entegra_select["product_name"].dtype)

object


In [1822]:
print(df_entegra_select["latitude"].dtype)

float64


In [1823]:
df_entegra_select["city"].nunique()

195

In [1824]:
# df_entegra_select.to_excel(fr"{output_path}\Entegra Sales City Location For Tableau (01.01.2024-11.07.2024).xlsx", index=False)  # Adoba..Data Folder

##### Sales Order Quantity

In [1833]:
df_entegra_select["status_name"].unique()

array(['Onaylandı', 'Yeni Siparis', 'İade-İptal'], dtype=object)

In [1834]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

['Trendyol', 'Hepsiburada', 'N11', 'Ciceksepeti', 'Woocommerce', 'Amazon']

In [1835]:
df_entegra_select_sales = df_entegra_select[(df_entegra_select["status_name"] == "Onaylandı") | (df_entegra_select["status_name"] == "Yeni Siparis")]
df_entegra_select_sales.reset_index(drop=True, inplace=True)
df_entegra_select_sales

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Yalova,Turkey,40.592898,29.077612
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,154.17,30.83,185.0,Trendyolekspress,Onaylandı,Bolu,Turkey,40.621210,31.646026
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,298.33,59.67,358.0,Trendyolekspress,Onaylandı,Agri,Turkey,39.777918,43.168437
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,300.00,60.00,360.0,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92429,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kütahya,Turkey,39.252251,29.493773
92430,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,3,Trendyol,318.18,31.82,350.0,Trendyolekspress,Onaylandı,Adiyaman,Turkey,37.763750,38.276348
92431,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,1,Trendyol,312.50,62.50,375.0,Trendyolekspress,Onaylandı,Aydın,Turkey,37.837948,27.845691
92432,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kocaeli,Turkey,40.821654,29.950718


In [1836]:
entegration_list_sales = list(df_entegra_select_sales["entegration"].unique())
entegration_list_sales

['Trendyol', 'Hepsiburada', 'N11', 'Ciceksepeti', 'Woocommerce', 'Amazon']

In [1837]:
df_sales_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_sales)
df_sales_quantity_entegration = df_sales_quantity_entegration.fillna(0)
df_sales_quantity_entegration.reset_index(inplace=True)
df_sales_quantity_entegration

entegration,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.0,0.0,1.0,0.0,0.0,0.0
1,000037,0.0,0.0,2.0,0.0,0.0,0.0
2,000571,0.0,1.0,5.0,0.0,0.0,0.0
3,000580,0.0,0.0,4.0,0.0,0.0,0.0
4,000622,0.0,2.0,4.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0
1032,Nan,0.0,0.0,0.0,0.0,0.0,0.0
1033,Sole,0.0,0.0,0.0,0.0,1.0,0.0
1034,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0


In [1838]:
df_sales_quantity_entegration = df_sales_quantity_entegration.rename_axis(None, axis=1)  # del index name (entegration)
df_sales_quantity_entegration

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.0,0.0,1.0,0.0,0.0,0.0
1,000037,0.0,0.0,2.0,0.0,0.0,0.0
2,000571,0.0,1.0,5.0,0.0,0.0,0.0
3,000580,0.0,0.0,4.0,0.0,0.0,0.0
4,000622,0.0,2.0,4.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0
1032,Nan,0.0,0.0,0.0,0.0,0.0,0.0
1033,Sole,0.0,0.0,0.0,0.0,1.0,0.0
1034,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0


In [1839]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_sales_quantity_entegration

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.0,0.0,1.0,0.0,0.0,0.0
1,000037,0.0,0.0,2.0,0.0,0.0,0.0
2,000571,0.0,1.0,5.0,0.0,0.0,0.0
3,000580,0.0,0.0,4.0,0.0,0.0,0.0
4,000622,0.0,2.0,4.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0
1032,Nan,0.0,0.0,0.0,0.0,0.0,0.0
1033,Sole,0.0,0.0,0.0,0.0,1.0,0.0
1034,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0


In [1840]:
df_sales_quantity_entegration["toplam_satış_adet"] = 0
for entg_column in entegration_list_all:
    df_sales_quantity_entegration["toplam_satış_adet"] += df_sales_quantity_entegration[f"{entg_column}"]

df_sales_quantity_entegration 

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,000037,0.0,0.0,2.0,0.0,0.0,0.0,2.0
2,000571,0.0,1.0,5.0,0.0,0.0,0.0,6.0
3,000580,0.0,0.0,4.0,0.0,0.0,0.0,4.0
4,000622,0.0,2.0,4.0,1.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1032,Nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1033,Sole,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1034,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0,2724.0


In [1841]:
df_sales_name_quantity_entegration  = pd.merge(df_sales_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_sales_name_quantity_entegration.drop_duplicates(inplace=True)
df_sales_name_quantity_entegration

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet,product_name
0,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...
1,000037,0.0,0.0,2.0,0.0,0.0,0.0,2.0,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...
2,000571,0.0,1.0,5.0,0.0,0.0,0.0,6.0,"Classone Tl6005 Workout 15,6 Inç Uyumlu Su Geç..."
3,000580,0.0,0.0,4.0,0.0,0.0,0.0,4.0,Classone Pr-R405 Genoa 15.6 Inç Uyumlu Noteboo...
4,000622,0.0,2.0,4.0,1.0,0.0,0.0,7.0,Classone Wg100 Gaming Kablosuz Mouse Siyah
...,...,...,...,...,...,...,...,...,...
1078,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0,2.0,Sarar Sürpriz Ev Tekstil Antrasit Gri Post Pel...
1079,Nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Nan
1080,Sole,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sole 750 W Mini Tost Makinesi
1081,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0,2724.0,Tanımsız


In [1842]:
cols = list(df_sales_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration[cols]
df_sales_name_quantity_entegration

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000037,0.0,0.0,2.0,0.0,0.0,0.0,2.0
2,"Classone Tl6005 Workout 15,6 Inç Uyumlu Su Geç...",000571,0.0,1.0,5.0,0.0,0.0,0.0,6.0
3,Classone Pr-R405 Genoa 15.6 Inç Uyumlu Noteboo...,000580,0.0,0.0,4.0,0.0,0.0,0.0,4.0
4,Classone Wg100 Gaming Kablosuz Mouse Siyah,000622,0.0,2.0,4.0,1.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...
1078,Sarar Sürpriz Ev Tekstil Antrasit Gri Post Pel...,Muk-Bgm6571,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1079,Nan,Nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1080,Sole 750 W Mini Tost Makinesi,Sole,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1081,Tanımsız,Tanımsız.,96.0,3.0,939.0,229.0,907.0,550.0,2724.0


In [1843]:
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration.sort_values(by="toplam_satış_adet", ascending=False)
df_sales_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
2,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0
3,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
4,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
...,...,...,...,...,...,...,...,...,...
1078,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1079,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1080,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1081,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [1844]:
df_sales_name_quantity_entegration_drop = df_sales_name_quantity_entegration.drop_duplicates(subset="model")
df_sales_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration_drop

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,0.0,0.0,219.0,0.0,4459.0,0.0,4678.0
...,...,...,...,...,...,...,...,...,...
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1034,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [1845]:
df_sales_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet.xlsx", index=False)

##### Sales Order TL

In [1846]:
df_entegra_select_sales

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,2024-07-11,16:21:44,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Yalova,Turkey,40.592898,29.077612
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,2024-07-11,16:20:54,1,Trendyol,154.17,30.83,185.0,Trendyolekspress,Onaylandı,Bolu,Turkey,40.621210,31.646026
2,003954,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,2024-07-11,16:20:05,1,Trendyol,298.33,59.67,358.0,Trendyolekspress,Onaylandı,Agri,Turkey,39.777918,43.168437
3,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2024-07-11,16:19:57,2,Trendyol,300.00,60.00,360.0,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,2024-07-11,16:19:42,1,Trendyol,383.33,76.67,460.0,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92429,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,2024-01-01,00:18:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kütahya,Turkey,39.252251,29.493773
92430,002979,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,2024-01-01,00:14:43,3,Trendyol,318.18,31.82,350.0,Trendyolekspress,Onaylandı,Adiyaman,Turkey,37.763750,38.276348
92431,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,2024-01-01,00:02:03,1,Trendyol,312.50,62.50,375.0,Trendyolekspress,Onaylandı,Aydın,Turkey,37.837948,27.845691
92432,002583,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,2024-01-01,00:02:00,1,N11,153.64,15.36,169.0,Mng,Yeni Siparis,Kocaeli,Turkey,40.821654,29.950718


In [1847]:
df_entegration_model_sales = df_entegra_select_sales.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_sales = df_entegration_model_sales.reset_index()
df_entegration_model_sales.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_sales.reset_index(drop=True, inplace=True)
df_entegration_model_sales

,model,entegration,total,tax,grand_total
0,Tanımsız.,Woocommerce,262463.0415,52065.8303,314528.8712
1,003661,Woocommerce,197106.8755,39421.3745,236528.2500
2,003943,Woocommerce,155836.5917,30965.4721,186802.0638
3,002828,Woocommerce,106517.1235,21122.3954,127639.5189
4,003662,Woocommerce,104804.0851,20949.3649,125753.4500
...,...,...,...,...,...
1909,003532,Amazon,283.3200,56.6600,339.9800
1910,004003,Amazon,283.2300,56.6500,339.8800
1911,002260,Amazon,153.3300,30.6700,184.0000
1912,003765,Amazon,149.8400,29.9600,179.8000


###### Total

In [1848]:
df_sales_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_total = df_sales_entegration_total.fillna(0)
df_sales_entegration_total.reset_index(inplace=True)
df_sales_entegration_total = df_sales_entegration_total.rename_axis(None, axis=1)
df_sales_entegration_total

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,36.0667,0.00,0.00,0.0000
1,000037,0.00,0.00,72.1334,0.00,0.00,0.0000
2,000571,0.00,271.72,1630.3180,0.00,0.00,0.0000
3,000580,0.00,0.00,1389.8908,0.00,0.00,0.0000
4,000622,0.00,256.32,376.2900,128.17,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,666.66,0.0000
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.0000,0.00,201.67,0.0000
1034,Tanımsız.,32692.02,1239.73,495454.1609,42531.43,295847.69,262463.0415


In [1850]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_total

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,36.0667,0.00,0.00,0.0000
1,000037,0.00,0.00,72.1334,0.00,0.00,0.0000
2,000571,0.00,271.72,1630.3180,0.00,0.00,0.0000
3,000580,0.00,0.00,1389.8908,0.00,0.00,0.0000
4,000622,0.00,256.32,376.2900,128.17,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,666.66,0.0000
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.0000,0.00,201.67,0.0000
1034,Tanımsız.,32692.02,1239.73,495454.1609,42531.43,295847.69,262463.0415


In [1851]:
# 1 way
df_sales_entegration_total["toplam_net_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_total["toplam_net_satış_tl"] += df_sales_entegration_total[f"{entg_column}"]

df_sales_entegration_total 

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_net_satış_tl
0,000028,0.00,0.00,36.0667,0.00,0.00,0.0000,3.606670e+01
1,000037,0.00,0.00,72.1334,0.00,0.00,0.0000,7.213340e+01
2,000571,0.00,271.72,1630.3180,0.00,0.00,0.0000,1.902038e+03
3,000580,0.00,0.00,1389.8908,0.00,0.00,0.0000,1.389891e+03
4,000622,0.00,256.32,376.2900,128.17,0.00,0.0000,7.607800e+02
...,...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,666.66,0.0000,6.666600e+02
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000,0.000000e+00
1033,Sole,0.00,0.00,0.0000,0.00,201.67,0.0000,2.016700e+02
1034,Tanımsız.,32692.02,1239.73,495454.1609,42531.43,295847.69,262463.0415,1.130228e+06


In [1852]:
df_sales_entegration_total = df_sales_entegration_total.sort_values(by="toplam_net_satış_tl", ascending=False)
df_sales_entegration_total.reset_index(drop=True, inplace=True)
df_sales_entegration_total = df_sales_entegration_total[["model","toplam_net_satış_tl"]]
df_sales_entegration_total

,model,toplam_net_satış_tl
0,003722,2.144530e+06
1,004319,1.432895e+06
2,003661,1.293404e+06
3,002583,1.231818e+06
4,003362,1.202235e+06
...,...,...
1031,002841,5.831670e+01
1032,002104,5.247000e+01
1033,000873,4.333330e+01
1034,000028,3.606670e+01


###### Tax

In [1853]:
df_sales_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_tax = df_sales_entegration_tax.fillna(0)
df_sales_entegration_tax.reset_index(inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax.rename_axis(None, axis=1)
df_sales_entegration_tax

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,7.2133,0.00,0.00,0.0000
1,000037,0.00,0.00,14.4266,0.00,0.00,0.0000
2,000571,0.00,27.17,163.0320,0.00,0.00,0.0000
3,000580,0.00,0.00,138.9892,0.00,0.00,0.0000
4,000622,0.00,51.26,66.3400,25.63,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,133.34,0.0000
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.0000,0.00,40.33,0.0000
1034,Tanımsız.,6538.51,123.97,99080.1291,8445.60,57635.58,52065.8303


In [1854]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_tax

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,7.2133,0.00,0.00,0.0000
1,000037,0.00,0.00,14.4266,0.00,0.00,0.0000
2,000571,0.00,27.17,163.0320,0.00,0.00,0.0000
3,000580,0.00,0.00,138.9892,0.00,0.00,0.0000
4,000622,0.00,51.26,66.3400,25.63,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,133.34,0.0000
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.0000,0.00,40.33,0.0000
1034,Tanımsız.,6538.51,123.97,99080.1291,8445.60,57635.58,52065.8303


In [1855]:
# 1 way
df_sales_entegration_tax["toplam_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_tax["toplam_vergi_tl"] += df_sales_entegration_tax[f"{entg_column}"]

df_sales_entegration_tax

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_vergi_tl
0,000028,0.00,0.00,7.2133,0.00,0.00,0.0000,7.2133
1,000037,0.00,0.00,14.4266,0.00,0.00,0.0000,14.4266
2,000571,0.00,27.17,163.0320,0.00,0.00,0.0000,190.2020
3,000580,0.00,0.00,138.9892,0.00,0.00,0.0000,138.9892
4,000622,0.00,51.26,66.3400,25.63,0.00,0.0000,143.2300
...,...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.0000,0.00,133.34,0.0000,133.3400
1032,Nan,0.00,0.00,0.0000,0.00,0.00,0.0000,0.0000
1033,Sole,0.00,0.00,0.0000,0.00,40.33,0.0000,40.3300
1034,Tanımsız.,6538.51,123.97,99080.1291,8445.60,57635.58,52065.8303,223889.6194


In [1856]:
df_sales_entegration_tax = df_sales_entegration_tax.sort_values(by="toplam_vergi_tl", ascending=False)
df_sales_entegration_tax.reset_index(drop=True, inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax[["model","toplam_vergi_tl"]]
df_sales_entegration_tax

,model,toplam_vergi_tl
0,003722,421409.0798
1,004319,284657.4574
2,003661,258668.8924
3,003362,229480.4439
4,Tanımsız.,223889.6194
...,...,...
1031,002754,0.7400
1032,002104,0.5200
1033,002914,0.0000
1034,003041,0.0000


###### Grand Total

In [1857]:
df_sales_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.fillna(0)
df_sales_entegration_grand_total.reset_index(inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.rename_axis(None, axis=1)
df_sales_entegration_grand_total

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,43.28,0.00,0.00,0.0000
1,000037,0.00,0.00,86.56,0.00,0.00,0.0000
2,000571,0.00,298.89,1793.35,0.00,0.00,0.0000
3,000580,0.00,0.00,1528.88,0.00,0.00,0.0000
4,000622,0.00,307.58,442.63,153.80,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.00,0.00,800.00,0.0000
1032,Nan,0.00,0.00,0.00,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.00,0.00,242.00,0.0000
1034,Tanımsız.,39230.23,1363.70,594534.29,50976.94,353483.11,314528.8712


In [1858]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_grand_total

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce
0,000028,0.00,0.00,43.28,0.00,0.00,0.0000
1,000037,0.00,0.00,86.56,0.00,0.00,0.0000
2,000571,0.00,298.89,1793.35,0.00,0.00,0.0000
3,000580,0.00,0.00,1528.88,0.00,0.00,0.0000
4,000622,0.00,307.58,442.63,153.80,0.00,0.0000
...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.00,0.00,800.00,0.0000
1032,Nan,0.00,0.00,0.00,0.00,0.00,0.0000
1033,Sole,0.00,0.00,0.00,0.00,242.00,0.0000
1034,Tanımsız.,39230.23,1363.70,594534.29,50976.94,353483.11,314528.8712


In [1859]:
# 1 way
df_sales_entegration_grand_total["toplam_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_grand_total["toplam_brüt_satış_tl"] += df_sales_entegration_grand_total[f"{entg_column}"]

df_sales_entegration_grand_total

,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_brüt_satış_tl
0,000028,0.00,0.00,43.28,0.00,0.00,0.0000,4.328000e+01
1,000037,0.00,0.00,86.56,0.00,0.00,0.0000,8.656000e+01
2,000571,0.00,298.89,1793.35,0.00,0.00,0.0000,2.092240e+03
3,000580,0.00,0.00,1528.88,0.00,0.00,0.0000,1.528880e+03
4,000622,0.00,307.58,442.63,153.80,0.00,0.0000,9.040100e+02
...,...,...,...,...,...,...,...,...
1031,Muk-Bgm6571,0.00,0.00,0.00,0.00,800.00,0.0000,8.000000e+02
1032,Nan,0.00,0.00,0.00,0.00,0.00,0.0000,0.000000e+00
1033,Sole,0.00,0.00,0.00,0.00,242.00,0.0000,2.420000e+02
1034,Tanımsız.,39230.23,1363.70,594534.29,50976.94,353483.11,314528.8712,1.354117e+06


In [1860]:
df_sales_entegration_grand_total = df_sales_entegration_grand_total.sort_values(by="toplam_brüt_satış_tl", ascending=False)
df_sales_entegration_grand_total.reset_index(drop=True, inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total[["model","toplam_brüt_satış_tl"]]
df_sales_entegration_grand_total

,model,toplam_brüt_satış_tl
0,003722,2.565939e+06
1,004319,1.717552e+06
2,003661,1.552073e+06
3,003362,1.431715e+06
4,002583,1.358478e+06
...,...,...
1031,002841,6.998000e+01
1032,002104,5.299000e+01
1033,000873,5.200000e+01
1034,000028,4.328000e+01


In [1861]:
data_sales_frames = [df_sales_entegration_total, df_sales_entegration_tax, df_sales_entegration_grand_total]

In [1862]:
df_sales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_sales_frames)
df_sales_merged = df_sales_merged.drop_duplicates()
df_sales_merged = df_sales_merged.reset_index(drop=True)
df_sales_merged

,model,toplam_net_satış_tl,toplam_vergi_tl,toplam_brüt_satış_tl
0,003722,2.144530e+06,421409.0798,2.565939e+06
1,004319,1.432895e+06,284657.4574,1.717552e+06
2,003661,1.293404e+06,258668.8924,1.552073e+06
3,002583,1.231818e+06,126659.4270,1.358478e+06
4,003362,1.202235e+06,229480.4439,1.431715e+06
...,...,...,...,...
1031,002841,5.831670e+01,11.6633,6.998000e+01
1032,002104,5.247000e+01,0.5200,5.299000e+01
1033,000873,4.333330e+01,8.6667,5.200000e+01
1034,000028,3.606670e+01,7.2133,4.328000e+01


In [1863]:
df_quantity_sales_entegration_merge  = pd.merge(df_sales_name_quantity_entegration_drop, df_sales_merged, how="left", on="model")
df_quantity_sales_entegration_merge.drop_duplicates(inplace=True)
df_quantity_sales_entegration_merge

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet,toplam_net_satış_tl,toplam_vergi_tl,toplam_brüt_satış_tl
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0,1.231818e+06,126659.4270,1.358478e+06
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0,8.397484e+05,82587.3949,9.223358e+05
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0,8.249409e+05,161310.9999,9.862488e+05
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0,6.677695e+05,132072.2355,7.998412e+05
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,0.0,0.0,219.0,0.0,4459.0,0.0,4678.0,2.144530e+06,421409.0798,2.565939e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.754167e+02,35.0833,2.105000e+02
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.754200e+02,35.0800,2.105000e+02
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.333750e+02,66.6750,4.000500e+02
1034,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.606670e+01,7.2133,4.328000e+01


In [1864]:
df_quantity_sales_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün Satış Adet Ve TL.xlsx", index=False)

###### Sales Duplicated

In [1865]:
df_sales_dublicated_entegration = df_sales_name_quantity_entegration[df_sales_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_sales_dublicated_entegration

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
3,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
4,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
5,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0
...,...,...,...,...,...,...,...,...,...
658,Pembe Ahşap Oyuncak Buzdolabı Çg126,003817,0.0,0.0,0.0,0.0,7.0,0.0,7.0
748,"Çg113 Sarı Ahşap Çocuk Mutfağı Pembe, Ahşap Mu...",002416,0.0,0.0,3.0,0.0,0.0,1.0,4.0
750,"Çg113 Sarı Ahşap Çocuk Mutfağı Pembe, Ahşap Mu...",002416,0.0,0.0,3.0,0.0,0.0,1.0,4.0
953,"Çg80 Ahşap Çocuk Kelebek Makyaj Masası, Oyun H...",002425,0.0,0.0,0.0,0.0,2.0,0.0,2.0


In [1866]:
df_sales_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet Yenile.xlsx", index=False)

##### Cancel Order Quantity

In [1887]:
df_entegra_select["status_name"].unique()

array(['Onaylandı', 'Yeni Siparis', 'İade-İptal'], dtype=object)

In [1888]:
df_entegra_select_cancel = df_entegra_select[(df_entegra_select["status_name"] == "İade-İptal")]
df_entegra_select_cancel.reset_index(drop=True, inplace=True)
df_entegra_select_cancel

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003952,"Awox Mercan Kettle Pembe, Mercan Kettle 1,7 Li...",2024-07-11 00:43:35,2024-07-11,00:43:35,1,Trendyol,230.83,46.17,277.00,Trendyolekspress,İade-İptal,Kars,Turkey,40.455844,42.997953
1,003944,"Katlanabilir Bebek Yıkama Küvet Seti 7 Parça, ...",2024-07-10 18:59:25,2024-07-10,18:59:25,1,Trendyol,658.33,131.67,790.00,Trendyolekspress,İade-İptal,Adana,Turkey,37.143800,35.498409
2,003661,Kırmızı Kulplu Eğitici Ahşap Montessori Ledli ...,2024-07-10 18:42:29,2024-07-10,18:42:29,1,Trendyol,2041.67,408.33,2450.00,Trendyolekspress,İade-İptal,Izmir,Turkey,38.423743,27.142802
3,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-10 17:27:22,2024-07-10,17:27:22,1,Trendyol,154.17,30.83,185.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
4,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-10 17:23:27,2024-07-10,17:23:27,1,Trendyol,154.17,30.83,185.00,Trendyolekspress,İade-İptal,Izmir,Turkey,38.423743,27.142802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,003661,Kırmızı Kulplu Eğitici Ahşap Montessori Mutfak...,2024-01-01 19:27:16,2024-01-01,19:27:16,1,Trendyol,1250.00,250.00,1500.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
2396,002413,"Çg108 Siyah Ahşap Ledli Çocuk Mutfağı Siyah, A...",2024-01-01 17:05:12,2024-01-01,17:05:12,1,Trendyol,1833.33,366.67,2200.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
2397,001602,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",2024-01-01 15:59:00,2024-01-01,15:59:00,3,Trendyol,237.53,40.09,277.62,Trendyolekspress,İade-İptal,Balikesir,Turkey,39.653182,27.889650
2398,002857,Brillahome Petek Desenli Dijital Baskı Pamuk T...,2024-01-01 14:48:15,2024-01-01,14:48:15,1,Trendyol,150.00,15.00,165.00,Trendyolekspress,İade-İptal,Ankara,Turkey,39.920776,32.854050


In [1889]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

['Trendyol', 'Hepsiburada', 'N11', 'Ciceksepeti', 'Woocommerce', 'Amazon']

In [1890]:
entegration_list_cancel = list(df_entegra_select_cancel["entegration"].unique())
entegration_list_cancel

['Trendyol']

In [1891]:
df_cancel_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_cancel)
df_cancel_quantity_entegration = df_cancel_quantity_entegration.fillna(0)
df_cancel_quantity_entegration.reset_index(inplace=True)
df_cancel_quantity_entegration

entegration,model,Trendyol
0,000622,1
1,000688,4
2,000697,2
3,000699,2
4,000801,2
...,...,...
330,004374,6
331,004375,4
332,004388,2
333,004442,1


In [1892]:
df_cancel_quantity_entegration = df_cancel_quantity_entegration.rename_axis(None, axis=1)  # del index name
df_cancel_quantity_entegration

,model,Trendyol
0,000622,1
1,000688,4
2,000697,2
3,000699,2
4,000801,2
...,...,...
330,004374,6
331,004375,4
332,004388,2
333,004442,1


In [1893]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_quantity_entegration

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon
0,000622,1,0,0,0,0,0
1,000688,4,0,0,0,0,0
2,000697,2,0,0,0,0,0
3,000699,2,0,0,0,0,0
4,000801,2,0,0,0,0,0
...,...,...,...,...,...,...,...
330,004374,6,0,0,0,0,0
331,004375,4,0,0,0,0,0
332,004388,2,0,0,0,0,0
333,004442,1,0,0,0,0,0


In [1894]:
df_cancel_quantity_entegration["toplam_iptal_adet"] = df_cancel_quantity_entegration[entegration_list_all].sum(axis=1)

df_cancel_quantity_entegration

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet
0,000622,1,0,0,0,0,0,1
1,000688,4,0,0,0,0,0,4
2,000697,2,0,0,0,0,0,2
3,000699,2,0,0,0,0,0,2
4,000801,2,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...
330,004374,6,0,0,0,0,0,6
331,004375,4,0,0,0,0,0,4
332,004388,2,0,0,0,0,0,2
333,004442,1,0,0,0,0,0,1


In [1896]:
df_cancel_name_quantity_entegration  = pd.merge(df_cancel_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_cancel_name_quantity_entegration.drop_duplicates(inplace=True)
df_cancel_name_quantity_entegration

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet,product_name
0,000622,1,0,0,0,0,0,1,Classone Wg100 Gaming Kablosuz Mouse Siyah
1,000688,4,0,0,0,0,0,4,"Hobby Life Alaska Nofrost Saklama Kabı 0,65 Li..."
2,000697,2,0,0,0,0,0,2,"Hobby Life Alaska Nofrost Saklama Kabı 1,85 Li..."
3,000699,2,0,0,0,0,0,2,"Hobby Life Alaska Nofrost Saklama Kabı 1,85 Li..."
4,000801,2,0,0,0,0,0,2,Natureva Çörekotu Yağı 10 Ml
...,...,...,...,...,...,...,...,...,...
358,004374,6,0,0,0,0,0,6,Awox Kıyma Makinası Et Çekme Makinası Sucuk So...
359,004375,4,0,0,0,0,0,4,Awox Vıte Pe330 Kıyma Makinesi
360,004388,2,0,0,0,0,0,2,100 Adet Pratik Tazeliği Koruyan Tencere Ve Ta...
361,004442,1,0,0,0,0,0,1,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...


In [1898]:
cols = list(df_cancel_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration[cols]
df_cancel_name_quantity_entegration

,product_name,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet
0,Classone Wg100 Gaming Kablosuz Mouse Siyah,000622,1,0,0,0,0,0,1
1,"Hobby Life Alaska Nofrost Saklama Kabı 0,65 Li...",000688,4,0,0,0,0,0,4
2,"Hobby Life Alaska Nofrost Saklama Kabı 1,85 Li...",000697,2,0,0,0,0,0,2
3,"Hobby Life Alaska Nofrost Saklama Kabı 1,85 Li...",000699,2,0,0,0,0,0,2
4,Natureva Çörekotu Yağı 10 Ml,000801,2,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...
358,Awox Kıyma Makinası Et Çekme Makinası Sucuk So...,004374,6,0,0,0,0,0,6
359,Awox Vıte Pe330 Kıyma Makinesi,004375,4,0,0,0,0,0,4
360,100 Adet Pratik Tazeliği Koruyan Tencere Ve Ta...,004388,2,0,0,0,0,0,2
361,Bakır Görünümlü Çelik Sahan Seti Omlet Yumurta...,004442,1,0,0,0,0,0,1


In [1899]:
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration.sort_values(by="toplam_iptal_adet", ascending=False)
df_cancel_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration

,product_name,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet
0,Awox Dual Elektrikli Çay Makinesi Beyaz-Kırmızı,004227,276,0,0,0,0,0,276
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,267,0,0,0,0,0,267
2,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,267,0,0,0,0,0,267
3,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,239,0,0,0,0,0,239
4,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,198,0,0,0,0,0,198
...,...,...,...,...,...,...,...,...,...
358,"1300 Derece Profesyonel Pürmüz Kafa, Pürmüz Ba...",0034301,1,0,0,0,0,0,1
359,Çocuk Oyun Etkinlik Çalışma Masası Aktivite Ma...,003431,1,0,0,0,0,0,1
360,Eternity Fitilli Peluş Tek Kişilik Nevresim Ta...,003485,1,0,0,0,0,0,1
361,"Kedi Köpek Yuvarlak Yatak Peluş, Kedi Köpek Ya...",003489,1,0,0,0,0,0,1


In [1900]:
df_cancel_name_quantity_entegration_drop = df_cancel_name_quantity_entegration.drop_duplicates(subset="model")
df_cancel_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration_drop

,product_name,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet
0,Awox Dual Elektrikli Çay Makinesi Beyaz-Kırmızı,004227,276,0,0,0,0,0,276
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,267,0,0,0,0,0,267
2,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,239,0,0,0,0,0,239
3,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,198,0,0,0,0,0,198
4,"Awox Nova Kettle Beyaz, 2000 W 1.7 Lt Su Isıtı...",003953,155,0,0,0,0,0,155
...,...,...,...,...,...,...,...,...,...
330,"1300 Derece Profesyonel Pürmüz Kafa, Pürmüz Ba...",0034301,1,0,0,0,0,0,1
331,Çocuk Oyun Etkinlik Çalışma Masası Aktivite Ma...,003431,1,0,0,0,0,0,1
332,Eternity Fitilli Peluş Tek Kişilik Nevresim Ta...,003485,1,0,0,0,0,0,1
333,"Kedi Köpek Yuvarlak Yatak Peluş, Kedi Köpek Ya...",003489,1,0,0,0,0,0,1


In [1901]:
df_cancel_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet.xlsx", index=False)

##### Cancel Order TL

In [1902]:
df_entegra_select_cancel

,model,product_name,datetime,date,time,total_product_quantity,entegration,total,tax,grand_total,cargo_company,status_name,city,country,latitude,longitude
0,003952,"Awox Mercan Kettle Pembe, Mercan Kettle 1,7 Li...",2024-07-11 00:43:35,2024-07-11,00:43:35,1,Trendyol,230.83,46.17,277.00,Trendyolekspress,İade-İptal,Kars,Turkey,40.455844,42.997953
1,003944,"Katlanabilir Bebek Yıkama Küvet Seti 7 Parça, ...",2024-07-10 18:59:25,2024-07-10,18:59:25,1,Trendyol,658.33,131.67,790.00,Trendyolekspress,İade-İptal,Adana,Turkey,37.143800,35.498409
2,003661,Kırmızı Kulplu Eğitici Ahşap Montessori Ledli ...,2024-07-10 18:42:29,2024-07-10,18:42:29,1,Trendyol,2041.67,408.33,2450.00,Trendyolekspress,İade-İptal,Izmir,Turkey,38.423743,27.142802
3,001603,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",2024-07-10 17:27:22,2024-07-10,17:27:22,1,Trendyol,154.17,30.83,185.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
4,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-10 17:23:27,2024-07-10,17:23:27,1,Trendyol,154.17,30.83,185.00,Trendyolekspress,İade-İptal,Izmir,Turkey,38.423743,27.142802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,003661,Kırmızı Kulplu Eğitici Ahşap Montessori Mutfak...,2024-01-01 19:27:16,2024-01-01,19:27:16,1,Trendyol,1250.00,250.00,1500.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
2396,002413,"Çg108 Siyah Ahşap Ledli Çocuk Mutfağı Siyah, A...",2024-01-01 17:05:12,2024-01-01,17:05:12,1,Trendyol,1833.33,366.67,2200.00,Trendyolekspress,İade-İptal,Bursa,Turkey,39.989588,28.894467
2397,001602,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",2024-01-01 15:59:00,2024-01-01,15:59:00,3,Trendyol,237.53,40.09,277.62,Trendyolekspress,İade-İptal,Balikesir,Turkey,39.653182,27.889650
2398,002857,Brillahome Petek Desenli Dijital Baskı Pamuk T...,2024-01-01 14:48:15,2024-01-01,14:48:15,1,Trendyol,150.00,15.00,165.00,Trendyolekspress,İade-İptal,Ankara,Turkey,39.920776,32.854050


In [1903]:
df_entegration_model_cancel = df_entegra_select_cancel.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_cancel = df_entegration_model_cancel.reset_index()
df_entegration_model_cancel.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_cancel.reset_index(drop=True, inplace=True)
df_entegration_model_cancel

,model,entegration,total,tax,grand_total
0,004227,Trendyol,66621.30,13324.28,79945.58
1,003722,Trendyol,63528.94,12705.93,76234.87
2,003661,Trendyol,46635.03,8893.97,55529.00
3,004319,Trendyol,38530.87,7706.13,46237.00
4,003565,Trendyol,37055.01,7411.10,44466.11
...,...,...,...,...,...
330,002957,Trendyol,49.99,10.00,59.99
331,002491,Trendyol,46.50,9.30,55.80
332,002123,Trendyol,32.40,6.48,38.88
333,003066,Trendyol,29.17,5.83,35.00


###### İptal Satış

In [1904]:
df_cancel_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_total = df_cancel_entegration_total.fillna(0)
df_cancel_entegration_total.reset_index(inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total.rename_axis(None, axis=1)
df_cancel_entegration_total

,model,Trendyol
0,000622,128.17
1,000688,186.10
2,000697,126.24
3,000699,159.15
4,000801,193.68
...,...,...
330,004374,12058.34
331,004375,9500.00
332,004388,229.55
333,004442,416.67


In [1905]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_total

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon
0,000622,128.17,0,0,0,0,0
1,000688,186.10,0,0,0,0,0
2,000697,126.24,0,0,0,0,0
3,000699,159.15,0,0,0,0,0
4,000801,193.68,0,0,0,0,0
...,...,...,...,...,...,...,...
330,004374,12058.34,0,0,0,0,0
331,004375,9500.00,0,0,0,0,0
332,004388,229.55,0,0,0,0,0
333,004442,416.67,0,0,0,0,0


In [1906]:
# 1 way
df_cancel_entegration_total["toplam_iptal_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_total["toplam_iptal_satış_tl"] += df_cancel_entegration_total[f"{entg_column}"]

df_cancel_entegration_total 

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_satış_tl
0,000622,128.17,0,0,0,0,0,128.17
1,000688,186.10,0,0,0,0,0,186.10
2,000697,126.24,0,0,0,0,0,126.24
3,000699,159.15,0,0,0,0,0,159.15
4,000801,193.68,0,0,0,0,0,193.68
...,...,...,...,...,...,...,...,...
330,004374,12058.34,0,0,0,0,0,12058.34
331,004375,9500.00,0,0,0,0,0,9500.00
332,004388,229.55,0,0,0,0,0,229.55
333,004442,416.67,0,0,0,0,0,416.67


In [1907]:
df_cancel_entegration_total = df_cancel_entegration_total.sort_values(by="toplam_iptal_satış_tl", ascending=False)
df_cancel_entegration_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total[["model","toplam_iptal_satış_tl"]]
df_cancel_entegration_total

,model,toplam_iptal_satış_tl
0,004227,66621.30
1,003722,63528.94
2,003661,46635.03
3,004319,38530.87
4,003565,37055.01
...,...,...
330,002957,49.99
331,002491,46.50
332,002123,32.40
333,003066,29.17


###### İptal Vergi

In [1908]:
df_cancel_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_tax = df_cancel_entegration_tax.fillna(0)
df_cancel_entegration_tax.reset_index(inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax.rename_axis(None, axis=1)
df_cancel_entegration_tax

,model,Trendyol
0,000622,25.63
1,000688,37.22
2,000697,25.25
3,000699,31.83
4,000801,25.20
...,...,...
330,004374,2411.66
331,004375,1900.00
332,004388,30.45
333,004442,83.33


In [1909]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_tax

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon
0,000622,25.63,0,0,0,0,0
1,000688,37.22,0,0,0,0,0
2,000697,25.25,0,0,0,0,0
3,000699,31.83,0,0,0,0,0
4,000801,25.20,0,0,0,0,0
...,...,...,...,...,...,...,...
330,004374,2411.66,0,0,0,0,0
331,004375,1900.00,0,0,0,0,0
332,004388,30.45,0,0,0,0,0
333,004442,83.33,0,0,0,0,0


In [1910]:
# 1 way
df_cancel_entegration_tax["toplam_iptal_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_tax["toplam_iptal_vergi_tl"] += df_cancel_entegration_tax[f"{entg_column}"]

df_cancel_entegration_tax

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_vergi_tl
0,000622,25.63,0,0,0,0,0,25.63
1,000688,37.22,0,0,0,0,0,37.22
2,000697,25.25,0,0,0,0,0,25.25
3,000699,31.83,0,0,0,0,0,31.83
4,000801,25.20,0,0,0,0,0,25.20
...,...,...,...,...,...,...,...,...
330,004374,2411.66,0,0,0,0,0,2411.66
331,004375,1900.00,0,0,0,0,0,1900.00
332,004388,30.45,0,0,0,0,0,30.45
333,004442,83.33,0,0,0,0,0,83.33


In [1911]:
df_cancel_entegration_tax = df_cancel_entegration_tax.sort_values(by="toplam_iptal_vergi_tl", ascending=False)
df_cancel_entegration_tax.reset_index(drop=True, inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax[["model","toplam_iptal_vergi_tl"]]
df_cancel_entegration_tax

,model,toplam_iptal_vergi_tl
0,004227,13324.28
1,003722,12705.93
2,003661,8893.97
3,004319,7706.13
4,003565,7411.10
...,...,...
330,003600,0.00
331,004207,0.00
332,003629,0.00
333,003626,0.00


###### Brüt İptal

In [1912]:
df_cancel_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.fillna(0)
df_cancel_entegration_grand_total.reset_index(inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.rename_axis(None, axis=1)
df_cancel_entegration_grand_total

,model,Trendyol
0,000622,153.80
1,000688,223.32
2,000697,151.49
3,000699,190.98
4,000801,218.88
...,...,...
330,004374,14470.00
331,004375,11400.00
332,004388,260.00
333,004442,500.00


In [1913]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_grand_total

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon
0,000622,153.80,0,0,0,0,0
1,000688,223.32,0,0,0,0,0
2,000697,151.49,0,0,0,0,0
3,000699,190.98,0,0,0,0,0
4,000801,218.88,0,0,0,0,0
...,...,...,...,...,...,...,...
330,004374,14470.00,0,0,0,0,0
331,004375,11400.00,0,0,0,0,0
332,004388,260.00,0,0,0,0,0
333,004442,500.00,0,0,0,0,0


In [1914]:
# 1 way
df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] += df_cancel_entegration_grand_total[f"{entg_column}"]

df_cancel_entegration_grand_total

,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_brüt_satış_tl
0,000622,153.80,0,0,0,0,0,153.80
1,000688,223.32,0,0,0,0,0,223.32
2,000697,151.49,0,0,0,0,0,151.49
3,000699,190.98,0,0,0,0,0,190.98
4,000801,218.88,0,0,0,0,0,218.88
...,...,...,...,...,...,...,...,...
330,004374,14470.00,0,0,0,0,0,14470.00
331,004375,11400.00,0,0,0,0,0,11400.00
332,004388,260.00,0,0,0,0,0,260.00
333,004442,500.00,0,0,0,0,0,500.00


In [1915]:
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.sort_values(by="toplam_iptal_brüt_satış_tl", ascending=False)
df_cancel_entegration_grand_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total[["model","toplam_iptal_brüt_satış_tl"]]
df_cancel_entegration_grand_total

,model,toplam_iptal_brüt_satış_tl
0,004227,79945.58
1,003722,76234.87
2,003661,55529.00
3,004319,46237.00
4,003565,44466.11
...,...,...
330,002957,59.99
331,002491,55.80
332,002123,38.88
333,003066,35.00


In [1916]:
data_cancel_frames = [df_cancel_entegration_total, df_cancel_entegration_tax, df_cancel_entegration_grand_total]

In [1917]:
df_cancel_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_cancel_frames)
df_cancel_merged = df_cancel_merged.drop_duplicates()
df_cancel_merged = df_cancel_merged.reset_index(drop=True)
df_cancel_merged

,model,toplam_iptal_satış_tl,toplam_iptal_vergi_tl,toplam_iptal_brüt_satış_tl
0,004227,66621.30,13324.28,79945.58
1,003722,63528.94,12705.93,76234.87
2,003661,46635.03,8893.97,55529.00
3,004319,38530.87,7706.13,46237.00
4,003565,37055.01,7411.10,44466.11
...,...,...,...,...
330,002957,49.99,10.00,59.99
331,002491,46.50,9.30,55.80
332,002123,32.40,6.48,38.88
333,003066,29.17,5.83,35.00


In [1918]:
df_quantity_cancel_entegration_merge  = pd.merge(df_cancel_name_quantity_entegration_drop, df_cancel_merged, how="left", on="model")
df_quantity_cancel_entegration_merge.drop_duplicates(inplace=True)
df_quantity_cancel_entegration_merge

,product_name,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet,toplam_iptal_satış_tl,toplam_iptal_vergi_tl,toplam_iptal_brüt_satış_tl
0,Awox Dual Elektrikli Çay Makinesi Beyaz-Kırmızı,004227,276,0,0,0,0,0,276,66621.30,13324.28,79945.58
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,267,0,0,0,0,0,267,36175.50,3727.57,39903.07
2,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,239,0,0,0,0,0,239,24050.80,2353.74,26404.54
3,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,198,0,0,0,0,0,198,26239.46,5150.05,31389.41
4,"Awox Nova Kettle Beyaz, 2000 W 1.7 Lt Su Isıtı...",003953,155,0,0,0,0,0,155,28903.34,5780.66,34684.00
...,...,...,...,...,...,...,...,...,...,...,...,...
330,"1300 Derece Profesyonel Pürmüz Kafa, Pürmüz Ba...",0034301,1,0,0,0,0,0,1,183.33,36.67,220.00
331,Çocuk Oyun Etkinlik Çalışma Masası Aktivite Ma...,003431,1,0,0,0,0,0,1,1041.67,208.33,1250.00
332,Eternity Fitilli Peluş Tek Kişilik Nevresim Ta...,003485,1,0,0,0,0,0,1,1181.82,118.18,1300.00
333,"Kedi Köpek Yuvarlak Yatak Peluş, Kedi Köpek Ya...",003489,1,0,0,0,0,0,1,365.90,36.59,402.49


In [1919]:
df_quantity_cancel_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün İptal Satış Adet Ve TL.xlsx", index=False)

###### Cancel Duplicated

In [1920]:
df_cancel_dublicated_entegration = df_cancel_name_quantity_entegration[df_cancel_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_cancel_dublicated_entegration

,product_name,model,Trendyol,Hepsiburada,N11,Ciceksepeti,Woocommerce,Amazon,toplam_iptal_adet
1,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",002583,267,0,0,0,0,0,267
2,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,267,0,0,0,0,0,267
4,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,198,0,0,0,0,0,198
5,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",001602,198,0,0,0,0,0,198
8,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,123,0,0,0,0,0,123
9,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",001603,123,0,0,0,0,0,123
12,Awox Sparkling Kahve Makinesi Krom,003723,73,0,0,0,0,0,73
13,Awox Sparkling Kahve Makinesi Mat Krom,003723,73,0,0,0,0,0,73
17,4'Lü Çok Amaçlı Dolap Içi Ve Çekmece Içi Düzen...,003172,61,0,0,0,0,0,61
18,Alas 4'Lü Çok Amaçlı Dolap Içi Ve Çekmece Içi ...,003172,61,0,0,0,0,0,61


In [1921]:
df_cancel_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet Yenile.xlsx", index=False)

##### Add Cancel Order To Sales

###### Part 1

In [1931]:
df_sales_name_quantity_entegration_drop

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,0.0,0.0,219.0,0.0,4459.0,0.0,4678.0
...,...,...,...,...,...,...,...,...,...
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1034,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [1932]:
df_cancel_name_quantity_entegration_drop_var = df_cancel_name_quantity_entegration_drop[["model","toplam_iptal_adet"]]
df_cancel_name_quantity_entegration_drop_var

,model,toplam_iptal_adet
0,004227,276
1,002583,267
2,002941,239
3,001602,198
4,003953,155
...,...,...
330,0034301,1
331,003431,1
332,003485,1
333,003489,1


In [1933]:
df_sales_cancel_quantity_all = pd.merge(df_sales_name_quantity_entegration_drop, df_cancel_name_quantity_entegration_drop_var, how="left", on="model")
df_sales_cancel_quantity_all.fillna(0, inplace=True)
df_sales_cancel_quantity_all

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet,toplam_iptal_adet
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0,267.0
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0,239.0
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0,198.0
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0,123.0
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,0.0,0.0,219.0,0.0,4459.0,0.0,4678.0,137.0
...,...,...,...,...,...,...,...,...,...,...
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1034,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [1934]:
df_sales_cancel_quantity_all.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Rakamları Adet Master.xlsx", index=False)

###### Part 2

In [1935]:
df_sales_merged

,model,toplam_net_satış_tl,toplam_vergi_tl,toplam_brüt_satış_tl
0,003722,2.144530e+06,421409.0798,2.565939e+06
1,004319,1.432895e+06,284657.4574,1.717552e+06
2,003661,1.293404e+06,258668.8924,1.552073e+06
3,002583,1.231818e+06,126659.4270,1.358478e+06
4,003362,1.202235e+06,229480.4439,1.431715e+06
...,...,...,...,...
1031,002841,5.831670e+01,11.6633,6.998000e+01
1032,002104,5.247000e+01,0.5200,5.299000e+01
1033,000873,4.333330e+01,8.6667,5.200000e+01
1034,000028,3.606670e+01,7.2133,4.328000e+01


In [1936]:
df_cancel_merged

,model,toplam_iptal_satış_tl,toplam_iptal_vergi_tl,toplam_iptal_brüt_satış_tl
0,004227,66621.30,13324.28,79945.58
1,003722,63528.94,12705.93,76234.87
2,003661,46635.03,8893.97,55529.00
3,004319,38530.87,7706.13,46237.00
4,003565,37055.01,7411.10,44466.11
...,...,...,...,...
330,002957,49.99,10.00,59.99
331,002491,46.50,9.30,55.80
332,002123,32.40,6.48,38.88
333,003066,29.17,5.83,35.00


In [1937]:
df_sales_cancel_quantity_tl_all = pd.merge(df_sales_cancel_quantity_all, df_sales_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2 = pd.merge(df_sales_cancel_quantity_tl_all, df_cancel_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2.fillna(0, inplace=True)
df_sales_cancel_quantity_tl_all2

,product_name,model,Amazon,Ciceksepeti,Hepsiburada,N11,Trendyol,Woocommerce,toplam_satış_adet,toplam_iptal_adet,toplam_net_satış_tl,toplam_vergi_tl,toplam_brüt_satış_tl,toplam_iptal_satış_tl,toplam_iptal_vergi_tl,toplam_iptal_brüt_satış_tl
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,0.0,7.0,1514.0,1178.0,5658.0,0.0,8357.0,267.0,1.231818e+06,126659.4270,1.358478e+06,36175.50,3727.57,39903.07
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,0.0,0.0,0.0,2.0,8105.0,45.0,8152.0,239.0,8.397484e+05,82587.3949,9.223358e+05,24050.80,2353.74,26404.54
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,0.0,234.0,219.0,193.0,5693.0,117.0,6456.0,198.0,8.249409e+05,161310.9999,9.862488e+05,26239.46,5150.05,31389.41
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,0.0,216.0,606.0,5.0,4336.0,32.0,5195.0,123.0,6.677695e+05,132072.2355,7.998412e+05,16331.69,3250.04,19581.70
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,0.0,0.0,219.0,0.0,4459.0,0.0,4678.0,137.0,2.144530e+06,421409.0798,2.565939e+06,63528.94,12705.93,76234.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.754167e+02,35.0833,2.105000e+02,0.00,0.00,0.00
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.754200e+02,35.0800,2.105000e+02,0.00,0.00,0.00
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.333750e+02,66.6750,4.000500e+02,375.00,75.00,450.00
1034,Pikas Sekreterlik Kapaklı Üstten Mekanizmalı D...,000028,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.606670e+01,7.2133,4.328000e+01,0.00,0.00,0.00


In [1938]:
df_sales_cancel_quantity_tl_all2.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Adet Ve TL Master.xlsx", index=False)